<a href="https://colab.research.google.com/github/AzDevops143/Ai-assignment/blob/main/AI_Assignment1_Search_and_CSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving input.txt to input.txt


In [2]:
def read_input(filename="input.txt"):
    with open(filename, "r") as f:
        lines = f.readlines()

    start = tuple(int(c) if c!='B' else 'B' for c in lines[0].split("=")[1].strip())
    goal  = tuple(int(c) if c!='B' else 'B' for c in lines[1].split("=")[1].strip())

    return start, goal

START, GOAL = read_input()
print("Start:", START)
print("Goal :", GOAL)


Start: (1, 2, 3, 'B', 4, 6, 7, 5, 8)
Goal : (1, 2, 3, 4, 5, 6, 7, 'B', 8)


In [3]:
def read_input(filename="input.txt"):
    with open(filename, "r") as f:
        lines = f.readlines()

    start = tuple(int(c) if c!='B' else 'B' for c in lines[0].split("=")[1].strip())
    goal  = tuple(int(c) if c!='B' else 'B' for c in lines[1].split("=")[1].strip())

    return start, goal

START, GOAL = read_input()
print("Start:", START)
print("Goal :", GOAL)


Start: (1, 2, 3, 'B', 4, 6, 7, 5, 8)
Goal : (1, 2, 3, 4, 5, 6, 7, 'B', 8)


In [5]:
import heapq, time, math, random
from collections import deque

goal_pos = {v:(i//3,i%3) for i,v in enumerate(GOAL)}

def get_neighbors(state):
    idx = state.index('B')
    x,y = idx//3, idx%3

    moves=[]
    for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
        nx,ny=x+dx,y+dy
        if 0<=nx<3 and 0<=ny<3:
            new_idx=nx*3+ny
            lst=list(state)
            lst[idx],lst[new_idx]=lst[new_idx],lst[idx]
            moves.append(tuple(lst))
    return moves


In [6]:
import heapq, time, math, random
from collections import deque

goal_pos = {v:(i//3,i%3) for i,v in enumerate(GOAL)}

def get_neighbors(state):
    idx = state.index('B')
    x,y = idx//3, idx%3

    moves=[]
    for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
        nx,ny=x+dx,y+dy
        if 0<=nx<3 and 0<=ny<3:
            new_idx=nx*3+ny
            lst=list(state)
            lst[idx],lst[new_idx]=lst[new_idx],lst[idx]
            moves.append(tuple(lst))
    return moves


In [7]:
def misplaced(state):
    return sum(1 for i,v in enumerate(state) if v!='B' and v!=GOAL[i])

def manhattan(state):
    dist=0
    for i,v in enumerate(state):
        if v=='B': continue
        x,y=i//3,i%3
        gx,gy=goal_pos[v]
        dist+=abs(x-gx)+abs(y-gy)
    return dist


In [8]:
def bfs(start):
    t0=time.time()

    q=deque([(start,[])])
    visited={start}
    explored=0

    while q:
        state,path=q.popleft()
        explored+=1

        if state==GOAL:
            return path, explored, time.time()-t0

        for nxt in get_neighbors(state):
            if nxt not in visited:
                visited.add(nxt)
                q.append((nxt,path+[nxt]))


In [9]:
path, explored, t = bfs(START)
print("BFS Success")
print("Steps:", len(path))
print("States explored:", explored)
print("Time:", t)


BFS Success
Steps: 2
States explored: 8
Time: 4.124641418457031e-05


In [11]:
def dfs(start,limit=30):
    t0=time.time()

    stack=[(start,[],0)]
    visited=set()
    explored=0

    while stack:
        state,path,depth=stack.pop()
        explored+=1

        if state==GOAL:
            return path, explored, time.time()-t0

        if depth>limit:
            continue

        visited.add(state)

        for nxt in get_neighbors(state):
            if nxt not in visited:
                stack.append((nxt,path+[nxt],depth+1))


In [12]:
path, explored, t = dfs(START)
print("DFS Success")
print("Steps:", len(path))
print("States explored:", explored)
print("Time:", t)


DFS Success
Steps: 30
States explored: 4405
Time: 0.009467840194702148


In [13]:
def astar(start,heuristic):
    t0=time.time()

    pq=[(heuristic(start),0,start,[])]
    visited=set()
    explored=0

    while pq:
        f,g,state,path=heapq.heappop(pq)
        explored+=1

        if state==GOAL:
            return path, explored, time.time()-t0

        if state in visited:
            continue
        visited.add(state)

        for nxt in get_neighbors(state):
            heapq.heappush(pq,(g+1+heuristic(nxt),g+1,nxt,path+[nxt]))


In [19]:
import heapq
import time
from itertools import count

# --- Configuration & Input ---
START = (1, 2, 3, 'B', 4, 6, 7, 5, 8)
GOAL = (1, 2, 3, 4, 5, 6, 7, 8, 'B')

# --- Heuristics ---
def misplaced(state):
    return sum(1 for s, g in zip(state, GOAL) if s != 'B' and s != g)

def manhattan(state):
    dist = 0
    for i, char in enumerate(state):
        if char != 'B':
            g_idx = GOAL.index(char)
            r1, c1 = divmod(i, 3)
            r2, c2 = divmod(g_idx, 3)
            dist += abs(r1 - r2) + abs(c1 - c2)
    return dist

# --- Core Algorithm ---
def get_neighbors(state):
    neighbors = []
    idx = state.index('B')
    r, c = divmod(idx, 3)
    # Define moves: Up, Down, Left, Right
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nr, nc = r + dr, c + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            n_idx = nr * 3 + nc
            new_s = list(state)
            new_s[idx], new_s[n_idx] = new_s[n_idx], new_s[idx]
            neighbors.append(tuple(new_s))
    return neighbors

def astar(start, heuristic_func):
    start_time = time.time()
    # Unique tie-breaker for each run to prevent comparison of tuples
    counter = count()

    # Priority Queue: (f_score, tie_breaker, g_score, current_state, path)
    pq = [(heuristic_func(start), next(counter), 0, start, [])]

    # Track the best g_score for each visited state
    visited = {start: 0}
    explored_count = 0

    while pq:
        f, _, g, current, path = heapq.heappop(pq)
        explored_count += 1

        if current == GOAL:
            return path, explored_count, time.time() - start_time

        for nxt in get_neighbors(current):
            new_g = g + 1
            # If we haven't seen this state or found a shorter path to it
            if nxt not in visited or new_g < visited[nxt]:
                visited[nxt] = new_g
                h = heuristic_func(nxt)
                heapq.heappush(pq, (new_g + h, next(counter), new_g, nxt, path + [nxt]))

    return None, explored_count, time.time() - start_time

# --- Execution ---
for name, func in [("Misplaced Tiles", misplaced), ("Manhattan Distance", manhattan)]:
    print(f"Running A* with {name}...")
    path, explored, t = astar(START, func)
    if path is not None:
        print(f"Steps: {len(path)} | Explored: {explored} | Time: {t:.4f}s")
    else:
        print("No solution found.")

Running A* with Misplaced Tiles...
Steps: 3 | Explored: 4 | Time: 0.0001s
Running A* with Manhattan Distance...
Steps: 3 | Explored: 4 | Time: 0.0001s


In [20]:
def simulated_annealing(start):
    current=start
    T=100
    steps=0

    while T>0.1:
        if current==GOAL:
            return current,steps

        neighbors=get_neighbors(current)
        next_state=random.choice(neighbors)

        delta=manhattan(next_state)-manhattan(current)

        if delta<0 or random.random()<math.exp(-delta/T):
            current=next_state

        T*=0.95
        steps+=1

    return current,steps


In [21]:
state,steps = simulated_annealing(START)
print("Final State:",state)
print("Steps:",steps)


Final State: (1, 2, 3, 4, 5, 6, 7, 8, 'B')
Steps: 23


In [22]:
slots=["S1","S2","S3","S4"]

domain={
"S1":["A","B","C"],
"S2":["A","B","C"],
"S3":["A","B","C"],
"S4":["A","B"]   # C cannot work Slot4
}


In [23]:
def valid(assign,slot,bot):
    for s,b in assign.items():
        if abs(int(s[1])-int(slot[1]))==1 and b==bot:
            return False
    return True

def select_unassigned(assign):
    remaining=[s for s in slots if s not in assign]
    return min(remaining,key=lambda s:len(domain[s]))

def backtrack(assign={}):
    if len(assign)==4:
        if len(set(assign.values()))==3:
            return assign
        return None

    var=select_unassigned(assign)

    for val in domain[var]:
        if valid(assign,var,val):
            assign[var]=val
            result=backtrack(assign)
            if result:
                return result
            del assign[var]
    return None


In [24]:
solution=backtrack({})
print("CSP Solution:",solution)


CSP Solution: {'S4': 'A', 'S1': 'A', 'S2': 'B', 'S3': 'C'}


In [25]:
with open("results.txt","w") as f:
    f.write("Assignment Executed Successfully")

files.download("results.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
!pip install fpdf2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 20.2 MB/s eta 0:00:00


In [26]:
with open("input.txt", "w") as f:
    # Use '+' to join strings into one argument
    f.write("Initial State\n"+"123\nB46\n758\n" + "Goal state\n" + "123\n456\n78B\n")

# Open the file in 'read' mode to verify
with open("input.txt", "r") as f:
    content = f.read()
    print("--- Contents of input.txt ---")
    print(content)

--- Contents of input.txt ---
Initial State
123
B46
758
Goal state
123
456
78B



In [24]:
"""
Manuscript Sorting Problem (8-puzzle variant)
Search Strategies: BFS, DFS, A* with Manhattan heuristic

Google Colab setup:
  # Cell 1: Install fpdf2
  !pip install fpdf2 -q

  # Cell 2: Upload input.txt (or create it) and run this script
  # Ensure input.txt is in current directory, or upload to /content/
"""

import os
import time
from collections import deque
import heapq
from fpdf import FPDF

# Goal state: 123/456/7B8
GOAL = "12345678B"

def normalize_state(s):
    """Ensure blank is 'B'; accept '0' as blank. Return 9-char string or None."""
    s = s.replace("0", "B")
    if len(s) != 9 or "B" not in s or s.count("B") != 1:
        return None
    valid = set("12345678B")
    if not all(c in valid for c in s):
        return None
    return s

def is_solvable(state_str, goal_str=GOAL):
    """8-puzzle: solvable iff inversion count (tiles 1-8) has same parity as goal."""
    def inversions(s):
        t = [c for c in s if c != "B"]
        n = len(t)
        count = 0
        for i in range(n):
            for j in range(i + 1, n):
                if t[i] > t[j]:
                    count += 1
        return count
    return (inversions(state_str) % 2) == (inversions(goal_str) % 2)

def parse_grid(lines):
    """Parse one 3x3 state from lines. Returns 9-char string or None."""
    grid = []
    for line in lines:
        line = line.strip().replace(";", " ").replace(":", " ")
        if not line:
            continue
        for part in line.split():
            part = part.strip()
            if len(part) == 3:
                grid.append(part)
            elif len(part) == 9:
                grid.extend([part[0:3], part[3:6], part[6:9]])
    if len(grid) >= 3:
        return (grid[0] + grid[1] + grid[2]).replace(" ", "")
    return None

def parse_input_file(lines):
    """Parse input.txt: first 3 rows = start, next 3 rows = goal (optional). Accept 0 or B for blank."""
    rows = []
    for line in lines:
        line = line.strip().replace(";", " ").replace(":", " ").replace(",", " ")
        for part in line.split():
            part = part.strip()
            if len(part) == 3:
                rows.append(part)
            elif len(part) == 9:
                rows.extend([part[0:3], part[3:6], part[6:9]])
    start = (rows[0] + rows[1] + rows[2]).replace(" ", "") if len(rows) >= 3 else None
    goal = (rows[3] + rows[4] + rows[5]).replace(" ", "") if len(rows) >= 6 else GOAL
    start = normalize_state(start) if start else None
    goal = normalize_state(goal) if goal else GOAL
    return start, goal

def state_to_string(state):
    """3x3 tuple/list to string of 9 chars."""
    if isinstance(state, str) and len(state) == 9:
        return state
    s = "".join(str(c) for c in state)
    return s

def string_to_grid(s):
    """String '12345678B' -> 3 rows for display."""
    return [s[0:3], s[3:6], s[6:9]]

def get_blank_pos(state):
    """Return index of 'B' (0-8)."""
    s = state_to_string(state)
    return s.index("B")

def get_successors(state):
    """Yield (action_name, new_state_string) for each valid move. Cost = 1."""
    s = state_to_string(state)
    i = get_blank_pos(s)
    row, col = i // 3, i % 3
    moves = [
        ("Up", -3),
        ("Down", 3),
        ("Left", -1),
        ("Right", 1),
    ]
    for name, delta in moves:
        j = i + delta
        if j < 0 or j > 8:
            continue
        if name in ("Left", "Right") and j // 3 != row:
            continue
        lst = list(s)
        lst[i], lst[j] = lst[j], lst[i]
        yield name, "".join(lst)

def is_goal(state):
    return state_to_string(state) == GOAL

def bfs(start):
    """BFS. Returns (path, states_explored) or (None, count)."""
    start_s = state_to_string(start)
    if is_goal(start_s):
        return [], 0
    parent = {start_s: (None, None)}
    q = deque([start_s])
    explored = 0
    while q:
        u = q.popleft()
        explored += 1
        if is_goal(u):
            path = []
            cur = u
            while parent[cur][0] is not None:
                path.append(parent[cur][1])
                cur = parent[cur][0]
            path.reverse()
            return path, explored
        for action, v in get_successors(u):
            if v not in parent:
                parent[v] = (u, action)
                q.append(v)
    return None, explored

def _dfs_limited(start_s, limit):
    """DFS with depth limit. Returns (path, count) or (None, count)."""
    stack = [(start_s, [], 0)]
    visited = {start_s}
    count = 0
    while stack:
        u, path, depth = stack.pop()
        count += 1
        if is_goal(u):
            return path, count
        if depth >= limit:
            continue
        for action, v in get_successors(u):
            if v not in visited:
                visited.add(v)
                stack.append((v, path + [action], depth + 1))
    return None, count

def dfs(start, max_depth=100):
    """Iterative deepening DFS so solution is found when solvable. Returns (path, states_explored)."""
    start_s = state_to_string(start)
    if is_goal(start_s):
        return [], 0
    total_explored = 0
    for depth_limit in range(1, max_depth + 1):
        path, count = _dfs_limited(start_s, depth_limit)
        total_explored += count
        if path is not None:
            return path, total_explored
    return None, total_explored

def manhattan_heuristic(state):
    """Sum of Manhattan distances of each tile (except B) to its goal position."""
    s = state_to_string(state)
    total = 0
    for i, c in enumerate(s):
        if c == "B":
            continue
        goal_idx = GOAL.index(c)
        r1, c1 = i // 3, i % 3
        r2, c2 = goal_idx // 3, goal_idx % 3
        total += abs(r1 - r2) + abs(c1 - c2)
    return total

def astar(start):
    """A* with Manhattan heuristic. Returns (path, states_explored) or (None, count)."""
    start_s = state_to_string(start)
    if is_goal(start_s):
        return [], 0
    # (f, g, state, path)
    g = 0
    h = manhattan_heuristic(start_s)
    open_set = [(g + h, g, start_s, [])]
    closed = set()
    explored = 0
    while open_set:
        f, g, u, path = heapq.heappop(open_set)
        explored += 1
        if u in closed:
            continue
        closed.add(u)
        if is_goal(u):
            return path, explored
        for action, v in get_successors(u):
            if v in closed:
                continue
            g_new = g + 1
            h_new = manhattan_heuristic(v)
            heapq.heappush(open_set, (g_new + h_new, g_new, v, path + [action]))
    return None, explored

def get_intermediate_states(start_s, path):
    """Given start state and path of moves, return list of (state_str, move_or_None)."""
    states = [(start_s, None)]
    current = start_s
    for move in path:
        for action, v in get_successors(current):
            if action == move:
                current = v
                states.append((current, action))
                break
    return states

def get_intermediate_states_with_ghf(start_s, path):
    """Return list of (state_str, move_or_None, g, h, f) for each step."""
    result = []
    current = start_s
    g = 0
    h = manhattan_heuristic(current)
    f = g + h
    result.append((current, None, g, h, f))
    for move in path:
        for action, v in get_successors(current):
            if action == move:
                current = v
                g += 1
                h = manhattan_heuristic(current)
                f = g + h
                result.append((current, action, g, h, f))
                break
    return result

def format_state_display(state_str):
    """Format 9-char state as 3x3 grid display."""
    return f"{state_str[0]} {state_str[1]} {state_str[2]}\n{state_str[3]} {state_str[4]} {state_str[5]}\n{state_str[6]} {state_str[7]} {state_str[8]}"

def run_search(name, start, search_fn, heuristic_name=None):
    """Run a search and return dict with Success/Failure, path, states, time."""
    t0 = time.perf_counter()
    path, states = search_fn(start)
    t1 = time.perf_counter()
    success = path is not None
    return {
        "success": success,
        "heuristic": heuristic_name,
        "path": path if path else [],
        "states": states,
        "time_sec": t1 - t0,
    }

def main():
    global GOAL
    # Colab: __file__ not defined; use cwd. Local: use script directory.
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()
    input_path = os.path.join(script_dir, "input.txt")
    try:
        with open(input_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
    except FileNotFoundError:
        print("Error: input.txt not found.")
        return

    start, goal = parse_input_file(lines)
    if goal:
        GOAL = goal

    if not start:
        print("Error: Invalid start state in input.txt. Use 3 rows like 123, 456, 78B (or 0 for blank)")
        return

    solvable = is_solvable(start, GOAL)
    if not solvable:
        results = [
            ("BFS", {"success": False, "heuristic": None, "path": [], "states": 0, "time_sec": 0.0}),
            ("DFS", {"success": False, "heuristic": "Iterative Deepening", "path": [], "states": 0, "time_sec": 0.0}),
            ("A*", {"success": False, "heuristic": "Manhattan Distance", "path": [], "states": 0, "time_sec": 0.0}),
        ]
    else:
        results = []
        results.append(("BFS", run_search("BFS", start, bfs)))
        results.append(("DFS", run_search("DFS", start, dfs, "Iterative Deepening")))
        results.append(("A*", run_search("A*", start, astar, "Manhattan Distance")))

    lines_for_pdf = [
        "Manuscript Sorting - Search Strategies Output",
        "",
        f"Start State: {start}",
        f"Goal State:  {GOAL}",
        "",
    ]
    for name, r in results:
        block = [
            f"Strategy: {name}",
            f"Success/Failure: {'Success' if r['success'] else 'Failure'}",
        ]
        if r["heuristic"]:
            block.append(f"Heuristic/Parameters used: {r['heuristic']}")
        path_str = " -> ".join(r["path"]) if r["path"] else "N/A"
        block.append(f"(Sub)Optimal Path: {path_str}")
        block.append(f"Total States Explored: {r['states']}")
        block.append(f"Total Time Taken: {r['time_sec']:.6f} seconds")
        if r["success"] and r["path"]:
            block.append("")
            block.append("Intermediate Stages (g(n), h(n), f(n)):")
            states_ghf = get_intermediate_states_with_ghf(start, r["path"])
            f_vals = []
            for i, (st, move, g_val, h_val, f_val) in enumerate(states_ghf):
                f_vals.append(f_val)
                lbl = "Start" if move is None else f"Step {i} ({move})"
                block.append(f"  {lbl}: g(n)={g_val}, h(n)={h_val}, f(n)={f_val}")
                for grid_row in format_state_display(st).split("\n"):
                    block.append(f"    {grid_row}")
                if i == len(states_ghf) - 1:
                    block.append(f"  Final f(n) = {f_val}")
                    eq = " + ".join(str(f) for f in f_vals) + " = " + str(sum(f_vals))
                    block.append(f"  Total sum of f(n) = {eq}")
        block.append("")
        lines_for_pdf.extend(block)
        print("Strategy:", name)
        print("Success/Failure:", "Success" if r["success"] else "Failure")
        if r["heuristic"]:
            print("Heuristic/Parameters used:", r["heuristic"])
        print("(Sub)Optimal Path:", path_str)
        print("Total States Explored:", r["states"])
        print("Total Time Taken:", f"{r['time_sec']:.6f} seconds")
        if r["success"] and r["path"]:
            print("Intermediate Stages (g(n), h(n), f(n)):")
            states_ghf = get_intermediate_states_with_ghf(start, r["path"])
            f_vals = []
            for i, (st, move, g_val, h_val, f_val) in enumerate(states_ghf):
                f_vals.append(f_val)
                lbl = "Start" if move is None else f"Step {i} ({move})"
                print(f"  {lbl}: g(n)={g_val}, h(n)={h_val}, f(n)={f_val}")
                print("   ", format_state_display(st).replace("\n", "\n    "))
                if i == len(states_ghf) - 1:
                    print(f"  Final f(n) = {f_val}")
                    eq = " + ".join(str(f) for f in f_vals) + " = " + str(sum(f_vals))
                    print(f"  Total sum of f(n) = {eq}")
        print()

    output_txt_path = os.path.join(script_dir, "output.txt")
    output_txt_lines = []
    for name, r in results:
        output_txt_lines.append(f"Strategy: {name}")
        output_txt_lines.append(f"Success/Failure: {'Success' if r['success'] else 'Failure'}")
        if r["heuristic"]:
            output_txt_lines.append(f"Heuristic/Parameters used: {r['heuristic']}")
        path_str = " -> ".join(r["path"]) if r["path"] else "N/A"
        output_txt_lines.append(f"(Sub)Optimal Path: {path_str}")
        output_txt_lines.append(f"Total States Explored: {r['states']}")
        output_txt_lines.append(f"Total Time Taken: {r['time_sec']:.6f} seconds")
        if r["success"] and r["path"]:
            output_txt_lines.append("")
            output_txt_lines.append("Intermediate Stages (g(n), h(n), f(n)):")
            states_ghf = get_intermediate_states_with_ghf(start, r["path"])
            f_vals = []
            for i, (st, move, g_val, h_val, f_val) in enumerate(states_ghf):
                f_vals.append(f_val)
                lbl = "Start" if move is None else f"Step {i} ({move})"
                output_txt_lines.append(f"  {lbl}: g(n)={g_val}, h(n)={h_val}, f(n)={f_val}")
                output_txt_lines.append("    " + format_state_display(st).replace("\n", "\n    "))
                if i == len(states_ghf) - 1:
                    output_txt_lines.append(f"  Final f(n) = {f_val}")
                    eq = " + ".join(str(f) for f in f_vals) + " = " + str(sum(f_vals))
                    output_txt_lines.append(f"  Total sum of f(n) = {eq}")
        output_txt_lines.append("")
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(output_txt_lines))
    print(f"Output saved to: {output_txt_path}")

    output_pdf_path = os.path.join(script_dir, "output.pdf")
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Helvetica", size=11)
    for line in lines_for_pdf:
        pdf.cell(0, 7, line, new_x="LMARGIN", new_y="NEXT")
    pdf.output(output_pdf_path)
    print(f"Output saved to: {output_pdf_path}")

if __name__ == "__main__":
    main()


Strategy: BFS
Success/Failure: Success
(Sub)Optimal Path: Right -> Down -> Right
Total States Explored: 17
Total Time Taken: 0.000145 seconds
Intermediate Stages (g(n), h(n), f(n)):
  Start: g(n)=0, h(n)=3, f(n)=3
    1 2 3
    B 4 6
    7 5 8
  Step 1 (Right): g(n)=1, h(n)=2, f(n)=3
    1 2 3
    4 B 6
    7 5 8
  Step 2 (Down): g(n)=2, h(n)=1, f(n)=3
    1 2 3
    4 5 6
    7 B 8
  Step 3 (Right): g(n)=3, h(n)=0, f(n)=3
    1 2 3
    4 5 6
    7 8 B
  Final f(n) = 3
  Total sum of f(n) = 3 + 3 + 3 + 3 = 12

Strategy: DFS
Success/Failure: Success
Heuristic/Parameters used: Iterative Deepening
(Sub)Optimal Path: Right -> Down -> Right
Total States Explored: 20
Total Time Taken: 0.000080 seconds
Intermediate Stages (g(n), h(n), f(n)):
  Start: g(n)=0, h(n)=3, f(n)=3
    1 2 3
    B 4 6
    7 5 8
  Step 1 (Right): g(n)=1, h(n)=2, f(n)=3
    1 2 3
    4 B 6
    7 5 8
  Step 2 (Down): g(n)=2, h(n)=1, f(n)=3
    1 2 3
    4 5 6
    7 B 8
  Step 3 (Right): g(n)=3, h(n)=0, f(n)=3
    1 2 3
  

In [25]:
from google.colab import files

# To download the text file
files.download('/content/output.txt')

# To download the PDF file
files.download('/content/output.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>